### Setup

In [1]:
// import dependencies

var moment = require('moment')
var { constants } = require('../utils')
var { advanceTimeAndBlock } = require('../utils/evm')
var { evm, common, contracts, bn } = require('../utils');
var { toUnit } = require('../utils/bn')
var { getLatestBlockTimestamp, getBlockTimestamp } = require('../utils/evm')
var { next, clear, bnToNumber } = require('../utils/jupyter')
var { Notebook } = require('../utils/notebook')
var { ethers } = require('hardhat')

var $ = new Notebook();

clear();

In [2]:
// setup tools and contracts

next(async () => {
    await $.setup(constants.FORK_BLOCK_NUMBER);

    eth_provider = await ethers.getSigner()
    usdc = await $.fetch('ERC20', constants.USDC_ADDRESS)
    usdc_provider = await $.impersonate(constants.RICH_USDC_ADDRESS);
    await contracts.setBalance(usdc_provider._address, toUnit(1000))    
    
    dai = await $.fetch('ERC20', constants.DAI_ADDRESS)
    dai_provider = await $.impersonate(constants.RICH_DAI_ADDRESS);
    await contracts.setBalance(dai_provider._address, toUnit(1000))    

    uniV3Pool = await $.fetch('IUniswapV3Pool', constants.UNISWAP_V3_ORACLE_POOL )
    uniV3Router = await $.fetch('ISwapRouter', constants.UNISWAP_V3_ROUTER )
    oracle = await $.deploy('Oracle',[])
});

In [3]:
next(async()=>{
    
    decimals_dai = await dai.decimals()
    decimals_usdc = await usdc.decimals()
    
    console.log('initial quote', 1.0001**(await oracle.consult(uniV3Pool.address, 1000))*(10**decimals_dai/10**decimals_usdc))
})

In [4]:
// charts configuration

next(async() =>{
    await $.resetRecording()
    $.resetTraces()
    
    $.addViewTrace(oracle, 'consult', [uniV3Pool.address, 140], 'Oracle TWAP')
    
    $.addBalanceTrace(dai, usdc_provider._address, 18, 'DAI Balance')
    $.addBalanceTrace(usdc, usdc_provider._address, 6, 'USDC Balance')    

    $.addEventTrace(uniV3Pool.web3, 'Swap')    
})

In [5]:
// making swaps

next(async()=>{
    
    await $.sleepAndExecute(
        $.time(300,'seconds'),
        $.time(15,'seconds'),[{
            run: async()=>{
                await common.makeASwap(
                    usdc_provider, 
                    usdc.address, 
                    dai.address, 
                    usdc_provider._address, 
                    500, 
                    100_000_000_000000 // 100M
                )
            console.log('quote', 1.0001**(await oracle.consult(uniV3Pool.address, 1000))*(10**decimals_dai/10**decimals_usdc))
            },
            every: 15
        }]
    )
    
    await $.draw()

})

initial quote 0.999602742756728
quote 0.999602742756728
quote 0.999602742756728
quote 1.8324838851952512
quote 3.5389831957344153
quote 7.030129560636673
quote 14.243060885134588
quote 29.289637516456395
quote 60.96482951274112
quote 128.19595221576185
